In [37]:
# Import dependencies
import pandas as pd
import numpy as np
import psycopg2
import config as creds
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

In [38]:
# Import data from AWS databse

def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

conn, cursor=connect()

Connected!


In [39]:
# Read in the data from the database
listings_df = pd.read_sql_query("select * from nyc_listings", conn)
listings_df

,id,name,host_id,host_name,borough,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review
0,77765,Superior @ Box House,417504,The Box House Hotel,Brooklyn,Greenpoint,40.737770,-73.953660,Hotel room,308,2,42,217,51
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.645290,-73.972380,Private room,299,30,9,356,1419
2,45910,Beautiful Queens Brownstone! - 5BR,204539,Mark,Queens,Ridgewood,40.703090,-73.899630,Entire home/apt,425,30,13,365,1030
3,45936,Couldn't Be Closer To Columbia Uni,867225,Rahul,Manhattan,Morningside Heights,40.806300,-73.959850,Private room,75,31,135,219,58
4,80493,Cozy room in East Village with AC,434987,Jennifer,Manhattan,East Village,40.723220,-73.986150,Private room,55,2,207,132,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31500,628769808856889664,Two bedroom apartment in Hoboken New Jersey.,14468718,Burak,Manhattan,West Village,40.740227,-74.027504,Entire home/apt,410,3,2,0,40
31501,27577588,Luxury Studio ON Grove Street E0C - B1CA,37412692,Kim,Manhattan,Ellis Island,40.718220,-74.037940,Entire home/apt,135,365,2,365,1087
31502,654151117629853651,Lovely 3- bedroom apartment,117540494,Miriam,Queens,Rosedale,40.647244,-73.720088,Entire home/apt,180,1,5,0,14
31503,553754115911961053,Trendy 3-bedroom apartment near Manhattan,15048320,India,Manhattan,Upper West Side,40.787320,-74.004470,Entire home/apt,240,5,18,152,16


In [40]:
listings_df=listings_df.drop(columns=['id', 'name', 'host_name', 'neighborhood', 'borough'])
listings_df.head()

,host_id,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review
0,417504,40.73777,-73.95366,Hotel room,308,2,42,217,51
1,2787,40.64529,-73.97238,Private room,299,30,9,356,1419
2,204539,40.70309,-73.89963,Entire home/apt,425,30,13,365,1030
3,867225,40.80630,-73.95985,Private room,75,31,135,219,58
4,434987,40.72322,-73.98615,Private room,55,2,207,132,25


In [42]:
num_reviews_host=listings_df.groupby('host_id')['number_of_reviews'].sum()
num_reviews_host.name='num_reviews_host'

In [11]:
num_listings=listings_df.groupby('host_id').size()
num_listings.name='num_listings'

In [43]:
listings_df=listings_df.join(num_listings, on='host_id', how='outer').join(num_reviews_host, on='host_id', how='outer')
listings_df.head()

,host_id,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review,num_listings,num_reviews_host
0,417504,40.73777,-73.95366,Hotel room,308,2,42,217,51,30,1366
10,417504,40.73756,-73.95350,Hotel room,529,2,10,100,298,30,1366
11,417504,40.73777,-73.95366,Private room,372,2,9,222,2,30,1366
13,417504,40.73777,-73.95366,Private room,372,2,2,222,4136,30,1366
22,417504,40.73777,-73.95366,Hotel room,308,2,22,221,88,30,1366


In [45]:
listings_df=listings_df.drop(columns=['host_id'])
listings_df.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review,num_listings,num_reviews_host
0,40.73777,-73.95366,Hotel room,308,2,42,217,51,30,1366
10,40.73756,-73.95350,Hotel room,529,2,10,100,298,30,1366
11,40.73777,-73.95366,Private room,372,2,9,222,2,30,1366
13,40.73777,-73.95366,Private room,372,2,2,222,4136,30,1366
22,40.73777,-73.95366,Hotel room,308,2,22,221,88,30,1366


In [46]:
cat_df=pd.get_dummies(listings_df, columns=['room_type'])
cat_df.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review,num_listings,num_reviews_host,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,40.73777,-73.95366,308,2,42,217,51,30,1366,0,1,0,0
10,40.73756,-73.95350,529,2,10,100,298,30,1366,0,1,0,0
11,40.73777,-73.95366,372,2,9,222,2,30,1366,0,0,1,0
13,40.73777,-73.95366,372,2,2,222,4136,30,1366,0,0,1,0
22,40.73777,-73.95366,308,2,22,221,88,30,1366,0,1,0,0


In [47]:
avg_lat, avg_long=cat_df[['latitude', 'longitude']].mean().values

In [48]:
cat_df['distance']=((cat_df['latitude']-avg_lat)**2+(cat_df['longitude']-avg_long)**2)**.5
cat_df

,latitude,longitude,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review,num_listings,num_reviews_host,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,distance
0,40.737770,-73.953660,308,2,42,217,51,30,1366,0,1,0,0,0.013570
10,40.737560,-73.953500,529,2,10,100,298,30,1366,0,1,0,0,0.013307
11,40.737770,-73.953660,372,2,9,222,2,30,1366,0,0,1,0,0.013570
13,40.737770,-73.953660,372,2,2,222,4136,30,1366,0,0,1,0,0.013570
22,40.737770,-73.953660,308,2,22,221,88,30,1366,0,1,0,0,0.013570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31499,40.655910,-73.998570,145,3,14,0,12,1,14,1,0,0,0,0.090181
31500,40.740227,-74.027504,410,3,2,0,40,1,2,1,0,0,0,0.084099
31502,40.647244,-73.720088,180,1,5,0,14,1,5,1,0,0,0,0.238298
31503,40.787320,-74.004470,240,5,18,152,16,1,18,1,0,0,0,0.084533


In [49]:
cat_df=cat_df.drop(columns=['latitude', 'longitude'])
cat_df

,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review,num_listings,num_reviews_host,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,distance
0,308,2,42,217,51,30,1366,0,1,0,0,0.013570
10,529,2,10,100,298,30,1366,0,1,0,0,0.013307
11,372,2,9,222,2,30,1366,0,0,1,0,0.013570
13,372,2,2,222,4136,30,1366,0,0,1,0,0.013570
22,308,2,22,221,88,30,1366,0,1,0,0,0.013570
...,...,...,...,...,...,...,...,...,...,...,...,...
31499,145,3,14,0,12,1,14,1,0,0,0,0.090181
31500,410,3,2,0,40,1,2,1,0,0,0,0.084099
31502,180,1,5,0,14,1,5,1,0,0,0,0.238298
31503,240,5,18,152,16,1,18,1,0,0,0,0.084533


In [50]:
cat_df

,price,minimum_nights,number_of_reviews,availability_365,days_since_last_review,num_listings,num_reviews_host,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,distance
0,308,2,42,217,51,30,1366,0,1,0,0,0.013570
10,529,2,10,100,298,30,1366,0,1,0,0,0.013307
11,372,2,9,222,2,30,1366,0,0,1,0,0.013570
13,372,2,2,222,4136,30,1366,0,0,1,0,0.013570
22,308,2,22,221,88,30,1366,0,1,0,0,0.013570
...,...,...,...,...,...,...,...,...,...,...,...,...
31499,145,3,14,0,12,1,14,1,0,0,0,0.090181
31500,410,3,2,0,40,1,2,1,0,0,0,0.084099
31502,180,1,5,0,14,1,5,1,0,0,0,0.238298
31503,240,5,18,152,16,1,18,1,0,0,0,0.084533


In [51]:
rf_valid=RandomForestRegressor()

In [52]:
X=cat_df.drop(columns=['price'])
y=cat_df['price']

In [53]:
from sklearn.preprocessing import StandardScaler

In [54]:
X_train, X_test, y_train, y_test=train_test_split(X, y)

In [55]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [56]:
rf_valid.fit(X_train_scaled, y_train)

RandomForestRegressor()

In [57]:
print(f'Train R2 Score: {rf_valid.score(X_train_scaled, y_train)}')
print(f'Test R2 Score: {rf_valid.score(X_test_scaled, y_test)}')

Train R2 Score: 0.8801147394896716
Test R2 Score: 0.052809774019669575


In [58]:
sorted(list(zip(rf_valid.feature_importances_, X.columns)), reverse=True)

[(0.2987295054538404, 'distance'),
 (0.2466197628633486, 'days_since_last_review'),
 (0.12037537359711707, 'availability_365'),
 (0.11997434647645955, 'num_reviews_host'),
 (0.08768081310495848, 'number_of_reviews'),
 (0.04136474465708409, 'num_listings'),
 (0.03624093493745726, 'minimum_nights'),
 (0.03243057013586565, 'room_type_Entire home/apt'),
 (0.013244716512916303, 'room_type_Private room'),
 (0.002035506186715306, 'room_type_Hotel room'),
 (0.001303726074237388, 'room_type_Shared room')]